In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
# Muat data yang sudah bersih dari terakhir
try:
    df = pd.read_csv('../Week2/df_rosaliarev_custom_stopwords.csv')
    df['final_content_v2'].fillna('', inplace=True)
    print("File berhasil dimuat.")
except FileNotFoundError:
    print("File tidak ditemukan. Pastikan Anda sudah menjalankan tahap sebelumnya.")

File berhasil dimuat.


C:\Users\Wede\AppData\Local\Temp\ipykernel_46540\1363887006.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['final_content_v2'].fillna('', inplace=True)


In [19]:
# Inisialisasi CountVectorizer
# CountVectorizer untuk membuat model Bag of Words
vectorizer = CountVectorizer()

In [20]:
# Buat matriks Bag of Words
# .fit_transform() akan mempelajari kosakata dan mengubah teks menjadi matriks angka
bow_matrix = vectorizer.fit_transform(df['final_content_v2'])

In [21]:
# Mendapatkan daftar semua kata (kosakata) sebagai nama kolom
feature_names = vectorizer.get_feature_names_out()

# Konversi ke DataFrame
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)

print("\n--- Representasi Bag of Words (10 Ulasan Pertama) ---")
display(bow_df.iloc[:10, :15])


--- Representasi Bag of Words (10 Ulasan Pertama) ---


,aamiiinnn,aamiin,aammin,aaya,abai,abal,abcd,abdet,abg,abis,abiss,abizzzz,ac,acak,acakacakan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# Simpan hasil Bag of Words ke CSV
BOW_FILE = 'bow_features.csv'
bow_df.to_csv(BOW_FILE, index=False)
print(f"\nMatriks Bag of Words berhasil dibuat dan disimpan sebagai '{BOW_FILE}'.")
print(f"Jumlah total ulasan (dokumen): {bow_matrix.shape[0]}")
print(f"Jumlah total kata unik (kosakata/fitur): {bow_matrix.shape[1]}")


Matriks Bag of Words berhasil dibuat dan disimpan sebagai 'bow_features.csv'.
Jumlah total ulasan (dokumen): 3928
Jumlah total kata unik (kosakata/fitur): 4477


In [23]:
# Muat file Bag of Words
try:
    # Analisis Awal
    num_reviews, initial_vocab_size = bow_df.shape
    print(f"Ukuran Kosakata Awal: {initial_vocab_size} kata unik.")

    # Identifikasi dan Hapus Kata-kata Langka (Rare Words Removal)
    # Menghitung total kemunculan setiap kata di seluruh ulasan
    word_counts = bow_df.sum(axis=0)

    # Menentukan kata-kata yang hanya muncul sekali (threshold = 1)
    rare_words_threshold = 1
    rare_words = word_counts[word_counts <= rare_words_threshold].index.tolist()

    print(f"\nDitemukan {len(rare_words)} kata yang hanya muncul sekali.")
    if rare_words:
        print("Contoh kata-kata yang akan dihapus (kemungkinan typo/sangat spesifik):")
        print(rare_words[:20]) # Menampilkan 20 contoh pertama

    # Melakukan proses penghapusan
    bow_df_cleaned = bow_df.drop(columns=rare_words)
    final_vocab_size = bow_df_cleaned.shape[1]
    
    # Menampilkan Hasil
    print("\n--- Hasil Pembersihan ---")
    print(f"Ukuran Kosakata Awal\t: {initial_vocab_size} kata")
    print(f"Ukuran Kosakata Akhir\t: {final_vocab_size} kata")
    print(f"Jumlah Kata yang Dihapus\t: {initial_vocab_size - final_vocab_size} kata")

    # Menyimpan hasil ke file baru
    CLEANED_BOW_FILE = 'bow_features_cleaned.csv'
    bow_df_cleaned.to_csv(CLEANED_BOW_FILE, index=False)
    print(f"\nFile BoW yang sudah lebih bersih disimpan sebagai '{CLEANED_BOW_FILE}'.")


except FileNotFoundError:
    print("Error: File 'bow_features.csv' tidak ditemukan. Pastikan file berada di direktori yang sama dengan notebook Anda.")
except Exception as e:
    print(f"Terjadi error: {e}")


Ukuran Kosakata Awal: 4477 kata unik.

Ditemukan 2791 kata yang hanya muncul sekali.
Contoh kata-kata yang akan dihapus (kemungkinan typo/sangat spesifik):
['aamiiinnn', 'aammin', 'aaya', 'abai', 'abcd', 'abg', 'abiss', 'abizzzz', 'acakacakan', 'acara', 'acc', 'acces', 'access', 'accont', 'actual', 'acungin', 'adaanya', 'adalagi', 'adamenemukan', 'adap']

--- Hasil Pembersihan ---
Ukuran Kosakata Awal	: 4477 kata
Ukuran Kosakata Akhir	: 1686 kata
Jumlah Kata yang Dihapus	: 2791 kata

File BoW yang sudah lebih bersih disimpan sebagai 'bow_features_cleaned.csv'.
